In [1]:
import torch
import pickle
with open('/common/home/apc120/Downloads/labelled/embeddings_test1.pkl', 'rb') as f1:
    test_emb = pickle.load(f1)
test_img_all = test_emb[0]
test_txt_all = test_emb[1]


with open('/common/home/apc120/Downloads/labelled/instructions_embeddings_test.pkl', 'rb') as f2:
    test_data_instr = pickle.load(f2)
test_txt_instr = test_data_instr[0]


with open('/common/home/apc120/Downloads/labelled/ingredients_embeddings_test.pkl', 'rb') as f3:
    test_data_ingr = pickle.load(f3)
test_txt_ingr = test_data_ingr[0]


with open('/common/home/apc120/Downloads/labelled/title_embeddings_test.pkl', 'rb') as f4:
    test_data_title = pickle.load(f4)
test_txt_title = test_data_title[0]

/common/home/apc120/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import numpy as np
import torch
def calculate_rank_recall(txt_data, img_data):
    results_dict = {}
    projection_txt, projection_img = txt_data, img_data
    
    idxs = range(1000)
    
    glob_rank = []
    glob_recall = {1:0.0,5:0.0,10:0.0}
    
    for i in range(10):
        ids = random.sample(range(0,txt_data.shape[0]-1), 1000)
        
        txt_sample = projection_txt[ids,:]
        img_sample = projection_img[ids,:]
        
        similarity = np.dot(txt_sample.cpu().numpy(), img_sample.T.cpu().numpy())

        med_rank = []
        
        recall = {1:0.0,5:0.0,10:0.0}
        
        for ii in idxs:
            # get a column of similarities
            sim = similarity[ii,:]
            # sort indices in descending order
            sorting = np.argsort(sim)[::-1].tolist()
            # find where the index of the pair sample ended up in the sorting
            pos = sorting.index(ii)  
            if (pos+1) == 1:
                recall[1]+=1
            if (pos+1) <=5:
                recall[5]+=1
            if (pos+1)<=10:
                recall[10]+=1
            # store the position
            med_rank.append(pos+1)
        for i in recall.keys():
            recall[i]=recall[i]/1000
        med = np.median(med_rank)
        for i in recall.keys():
            glob_recall[i]+=recall[i]
        glob_rank.append(med)

    for i in glob_recall.keys():
        glob_recall[i] = glob_recall[i]/10
    med_dict = {}
    med_dict["mean_median"] = np.average(glob_rank)
    med_dict["recall"] = glob_recall
    med_dict["median_all"] = glob_rank
    print("Result:",med_dict)
    return med_dict

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class TextEncoder(nn.Module):
    def __init__(self,output_size,input_size=1024):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.LeakyReLU()
        )
        self.output = nn.Linear(512, output_size)
    
    def forward(self, x):
        x = self.layers(x)
        return self.output(x)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class ImgEncoder(nn.Module):
    def __init__(self,output_size,input_size=1024):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(),
            nn.LeakyReLU()
        )
        self.output = nn.Linear(512, output_size)
    
    def forward(self, x):
        x = self.layers(x)
        return self.output(x)

<h1>All Text Data</h1>

In [8]:
img_model=torch.load('64_mse_best_img_model.pth')
txt_model = torch.load('64_mse_best_txt_model.pth')
with torch.no_grad():
    img_model.eval()
    txt_model.eval()    
    test_result = calculate_rank_recall(txt_model(torch.tensor(test_txt_all).to('cuda')),img_model(torch.tensor(test_img_all).to('cuda')))

Result: {'mean_median': 5.55, 'recall': {1: 0.2414, 5: 0.4939, 10: 0.603}, 'median_all': [6.0, 5.5, 6.0, 6.0, 5.0, 5.0, 6.0, 5.0, 6.0, 5.0]}


<h1>Instructions</h1>

In [10]:
img_model=torch.load('64instr_mse_best_img_model.pth')
txt_model = torch.load('64instr_mse_best_txt_model.pth')
with torch.no_grad():   
    img_model.eval()
    txt_model.eval()
    test_result = calculate_rank_recall(txt_model(torch.tensor(test_txt_instr).to('cuda')),img_model(torch.tensor(test_img_all).to('cuda')))

Result: {'mean_median': 32.0, 'recall': {1: 0.0824, 5: 0.22209999999999996, 10: 0.31410000000000005}, 'median_all': [36.0, 31.5, 30.0, 29.0, 35.0, 29.0, 36.5, 29.0, 32.0, 32.0]}


<h1>Ingredients</h1>

In [11]:
img_model=torch.load('64ingr_mse_best_img_model.pth')
txt_model = torch.load('64ingr_mse_best_txt_model.pth')
with torch.no_grad():    
    img_model.eval()
    txt_model.eval()
    test_result = calculate_rank_recall(txt_model(torch.tensor(test_txt_ingr).to('cuda')),img_model(torch.tensor(test_img_all).to('cuda')))

Result: {'mean_median': 38.55, 'recall': {1: 0.0654, 5: 0.18980000000000002, 10: 0.2757}, 'median_all': [41.0, 37.5, 45.0, 34.0, 39.0, 38.0, 35.0, 38.0, 36.0, 42.0]}


<h1>Title</h1>

In [13]:
img_model=torch.load('64title_mse_best_img_model.pth')
txt_model = torch.load('64title_mse_best_txt_model.pth')
with torch.no_grad():
    img_model.eval()
    txt_model.eval()
    test_result = calculate_rank_recall(txt_model(torch.tensor(test_txt_title).to('cuda')),img_model(torch.tensor(test_img_all).to('cuda')))

Result: {'mean_median': 111.7, 'recall': {1: 0.027200000000000002, 5: 0.09299999999999999, 10: 0.1507}, 'median_all': [117.0, 106.0, 126.0, 110.0, 127.0, 108.0, 122.0, 97.0, 96.5, 107.5]}
